<a href="https://colab.research.google.com/github/gemepolimi/gemepolimi/blob/main/Addis_and_GONDAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.4 MB/s eta 0:00:00


In [2]:
import pandas as pd

# List your files here
files_to_process = ["ADDIS_ABABA_BOLE_ET.xlsx", "GONDAR_ET.xlsx"]

def apply_professional_format(file_path):
    output_path = f"Clean_Report_{file_path}"
    # Read all sheets from the original file
    all_sheets = pd.read_excel(file_path, sheet_name=None)

    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        for sheet_name, df in all_sheets.items():
            # Standardizing data to 2 decimal places
            df = df.round(2)
            df.to_excel(writer, sheet_name=sheet_name, index=False)

            workbook  = writer.book
            worksheet = writer.sheets[sheet_name]

            # --- PROFESSIONAL STYLES ---
            # 1. Header: Dark Blue background, White Bold text
            header_fmt = workbook.add_format({
                'bold': True, 'font_color': 'white', 'bg_color': '#2F5597',
                'border': 1, 'align': 'center', 'valign': 'vcenter'
            })
            # 2. Totals: Light Blue background, Bold, 2 Decimals
            total_fmt = workbook.add_format({
                'bold': True, 'bg_color': '#DDEBF7', 'border': 1, 'num_format': '0.00'
            })
            # 3. Data Body: Standard borders and 2 Decimals
            data_fmt = workbook.add_format({'border': 1, 'num_format': '0.00'})

            # Apply format to the header row
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_fmt)

            # Apply format to the data cells
            for row in range(1, len(df) + 1):
                # Identify if this is the "Monthly Total" row (usually at the bottom)
                is_total_row = (row == len(df))
                current_fmt = total_fmt if is_total_row else data_fmt

                for col in range(len(df.columns)):
                    val = df.iloc[row-1, col]
                    worksheet.write(row, col, val, current_fmt)

            # Auto-adjust column width for readability
            worksheet.set_column(0, len(df.columns), 12)

    print(f"✅ Formatted report created: {output_path}")

# Run the process
for file in files_to_process:
    try:
        apply_professional_format(file)
    except Exception as e:
        print(f"❌ Error processing {file}: {e}")

❌ Error processing ADDIS_ABABA_BOLE_ET.xlsx: NAN/INF not supported in write_number() without 'nan_inf_to_errors' Workbook() option
❌ Error processing GONDAR_ET.xlsx: NAN/INF not supported in write_number() without 'nan_inf_to_errors' Workbook() option


In [3]:
!pip install xlsxwriter

import pandas as pd

# The exact files you mentioned
station_files = ["ADDIS_ABABA_BOLE_ET.xlsx", "GONDAR_ET.xlsx"]

def create_clean_report(file_path):
    print(f"Processing: {file_path}")

    # Load every single sheet from the workbook
    excel_data = pd.read_excel(file_path, sheet_name=None)
    output_filename = f"Professional_Clean_{file_path}"

    with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
        for year_sheet, df in excel_data.items():

            # 1. CLEANING: Replace all empty/NaN values with 0.00
            df = df.fillna(0).round(2)

            # Write data to the specific sheet
            df.to_excel(writer, sheet_name=str(year_sheet), index=False)

            workbook  = writer.book
            worksheet = writer.sheets[str(year_sheet)]

            # 2. STYLES (Based on your screenshots)
            # Green Header
            header_fmt = workbook.add_format({
                'bold': True, 'bg_color': '#C6E0B4', 'border': 1,
                'align': 'center', 'font_name': 'Calibri'
            })

            # Annual Total Column
            annual_col_fmt = workbook.add_format({
                'bg_color': '#FCE4D6', 'border': 1, 'num_format': '0.00'
            })

            # Monthly Total Row
            footer_fmt = workbook.add_format({
                'bold': True, 'border': 1, 'num_format': '0.00'
            })

            # Standard Data Cells
            data_fmt = workbook.add_format({
                'border': 1, 'num_format': '0.00'
            })

            # 3. APPLY FORMATTING
            # Headers
            for col_num, value in enumerate(df.columns.values):
                worksheet.write(0, col_num, value, header_fmt)

            # Body and Totals
            for row_num in range(1, len(df) + 1):
                is_last_row = (row_num == len(df))
                for col_num in range(len(df.columns)):
                    val = df.iloc[row_num-1, col_num]

                    if is_last_row:
                        worksheet.write(row_num, col_num, val, footer_fmt)
                    elif col_num == 13: # Total Annual Column
                        worksheet.write(row_num, col_num, val, annual_col_fmt)
                    else:
                        worksheet.write(row_num, col_num, val, data_fmt)

            # Set column widths for a clean look
            worksheet.set_column('A:N', 11)

    print(f"✅ Success! Report ready: {output_filename}")

# Run for both station files
for station_file in station_files:
    try:
        create_clean_report(station_file)
    except Exception as e:
        print(f"Error: {e}")

Processing: ADDIS_ABABA_BOLE_ET.xlsx
✅ Success! Report ready: Professional_Clean_ADDIS_ABABA_BOLE_ET.xlsx
Processing: GONDAR_ET.xlsx
✅ Success! Report ready: Professional_Clean_GONDAR_ET.xlsx


In [4]:
import pandas as pd

# Files to process
station_files = ["ADDIS_ABABA_BOLE_ET.xlsx", "GONDAR_ET.xlsx"]

def create_summary_report(file_path):
    print(f"Creating Summary for: {file_path}")

    # Read all sheets (years) from the original data
    excel_data = pd.read_excel(file_path, sheet_name=None)
    summary_list = []

    for year, df in excel_data.items():
        # Get the 'Monthly Total' row (the last row in your previous sheets)
        # We assume the last row has the sums for J, F, M...
        monthly_values = df.iloc[-1, 1:13].values # Columns J through D
        annual_total = df.iloc[-1, 13]            # Column 'Total Annual'

        # Create a row for this year
        row = [year] + list(monthly_values) + [annual_total]
        summary_list.append(row)

    # Create Summary DataFrame
    columns = ['Year', 'J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D', 'Total Annual']
    summary_df = pd.DataFrame(summary_list, columns=columns)

    # Calculate "Media General" (Average of all years)
    media_general = summary_df.mean(numeric_only=True).round(2).tolist()
    # Replace the 'Year' value in the list with the label
    media_general_row = ["Media General"] + media_general[1:]

    # Output file name
    output_filename = f"Summary_Report_{file_path}"

    with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
        summary_df.to_excel(writer, sheet_name='Summary', index=False)

        workbook  = writer.book
        worksheet = writer.sheets['Summary']

        # --- STYLES ---
        header_fmt = workbook.add_format({
            'bold': True, 'bg_color': '#C6E0B4', 'border': 1, 'align': 'center'
        })
        data_fmt = workbook.add_format({
            'border': 1, 'num_format': '0.00'
        })
        media_fmt = workbook.add_format({
            'bold': True, 'bg_color': '#D9D9D9', 'border': 1, 'num_format': '0.00'
        })

        # Apply Headers
        for col_num, value in enumerate(columns):
            worksheet.write(0, col_num, value, header_fmt)

        # Apply Data rows
        for row_num, row_data in enumerate(summary_list):
            for col_num, val in enumerate(row_data):
                worksheet.write(row_num + 1, col_num, val, data_fmt)

        # Apply "Media General" Row at the bottom
        last_row_idx = len(summary_list) + 1
        for col_num, val in enumerate(media_general_row):
            worksheet.write(last_row_idx, col_num, val, media_fmt)

        # Column sizing
        worksheet.set_column('A:N', 12)

    print(f"✅ Summary Saved: {output_filename}")

# Run for both
for f in station_files:
    try:
        create_summary_report(f)
    except Exception as e:
        print(f"Error: {e}")

Creating Summary for: ADDIS_ABABA_BOLE_ET.xlsx
✅ Summary Saved: Summary_Report_ADDIS_ABABA_BOLE_ET.xlsx
Creating Summary for: GONDAR_ET.xlsx
✅ Summary Saved: Summary_Report_GONDAR_ET.xlsx


In [5]:
!pip install xlsxwriter

import pandas as pd

# Define your exact station files
station_files = ["ADDIS_ABABA_BOLE_ET.xlsx", "GONDAR_ET.xlsx"]

def create_combined_professional_report(file_path):
    print(f"Processing combined report for: {file_path}")

    # Read all original sheets
    excel_data = pd.read_excel(file_path, sheet_name=None)
    output_filename = f"Final_Professional_Report_{file_path}"

    # --- 1. PREPARE THE SUMMARY DATA ---
    summary_list = []
    for year_sheet, df in excel_data.items():
        # Clean data: Replace NaNs with 0
        df = df.fillna(0).round(2)
        # Assuming the last row is 'Monthly Total'
        monthly_values = df.iloc[-1, 1:13].values
        annual_total = df.iloc[-1, 13]
        # Store year as a clean integer
        summary_list.append([int(float(year_sheet))] + list(monthly_values) + [annual_total])

    summary_cols = ['Year', 'J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D', 'Total Annual']
    summary_df = pd.DataFrame(summary_list, columns=summary_cols)

    # Calculate Media General
    media_general = summary_df.mean(numeric_only=True).round(2).tolist()
    media_general_row = ["Media General"] + media_general[1:]

    # --- 2. START WRITING THE WORKBOOK ---
    with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
        workbook = writer.book

        # Define Styles
        header_fmt = workbook.add_format({'bold': True, 'bg_color': '#C6E0B4', 'border': 1, 'align': 'center'})
        data_fmt = workbook.add_format({'border': 1, 'num_format': '0.00'})
        int_fmt = workbook.add_format({'border': 1, 'num_format': '0'}) # For clean integers (Days/Years)
        total_fmt = workbook.add_format({'bold': True, 'bg_color': '#DDEBF7', 'border': 1, 'num_format': '0.00'})
        media_fmt = workbook.add_format({'bold': True, 'bg_color': '#D9D9D9', 'border': 1, 'num_format': '0.00'})

        # A. Add Summary Sheet (First Sheet)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        summary_sheet = writer.sheets['Summary']

        for col_num, value in enumerate(summary_cols):
            summary_sheet.write(0, col_num, value, header_fmt)

        for row_num, row_data in enumerate(summary_list):
            summary_sheet.write(row_num + 1, 0, row_data[0], int_fmt) # Year as Integer
            for col_num, val in enumerate(row_data[1:], start=1):
                summary_sheet.write(row_num + 1, col_num, val, data_fmt)

        # Add Media General Row
        last_row_idx = len(summary_list) + 1
        summary_sheet.write(last_row_idx, 0, media_general_row[0], media_fmt)
        for col_num, val in enumerate(media_general_row[1:], start=1):
            summary_sheet.write(last_row_idx, col_num, val, media_fmt)
        summary_sheet.set_column('A:N', 12)

        # B. Add Daily Year Sheets
        for year_sheet, df in excel_data.items():
            df = df.fillna(0).round(2)
            sheet_name = str(int(float(year_sheet))) # Clean year string
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            ws = writer.sheets[sheet_name]

            # Formatting Daily Table
            for col_num, value in enumerate(df.columns.values):
                ws.write(0, col_num, value, header_fmt)

            for row_num in range(1, len(df) + 1):
                is_last_row = (row_num == len(df))
                row_fmt = total_fmt if is_last_row else data_fmt

                # Write Day Column as clean integer
                day_val = df.iloc[row_num-1, 0]
                if not is_last_row:
                    ws.write(row_num, 0, int(float(day_val)), int_fmt)
                else:
                    ws.write(row_num, 0, day_val, row_fmt) # Label "Monthly Total"

                # Write Data Columns
                for col_num in range(1, len(df.columns)):
                    val = df.iloc[row_num-1, col_num]
                    ws.write(row_num, col_num, val, row_fmt)

            ws.set_column('A:N', 11)

    print(f"✅ Final report ready: {output_filename}")

# Run for both stations
for station_file in station_files:
    try:
        create_combined_professional_report(station_file)
    except Exception as e:
        print(f"Error: {e}")

Processing combined report for: ADDIS_ABABA_BOLE_ET.xlsx
✅ Final report ready: Final_Professional_Report_ADDIS_ABABA_BOLE_ET.xlsx
Processing combined report for: GONDAR_ET.xlsx
✅ Final report ready: Final_Professional_Report_GONDAR_ET.xlsx
